# Лемматизация и очистка от контекстных стоп-слов

In [1]:
import pandas as pd
import numpy as np
import os
import re
import glob

from tqdm.notebook import tqdm

np.random.seed(10)

UNIVS = ['АлтГУ', 'АлтГТУ', 'ВолГУ', 'ИМСИТ']
CURRPATH = os.getcwd()
DATAPATH = '\\'.join(CURRPATH.split('\\')[:-1]) + '\\new-data'
FILESPATH = DATAPATH + '\\new-files'

# Документы, которые невозможно адекватно парсить:
IGNOREFILES = [
    '\\ИМСИТ\\педагогическое образование история и право 3++\\сопоставительная германистика.txt',
    '\\ИМСИТ\\педагогическое образование англ нем 3++\\культурология.txt',
    '\\ИМСИТ\\педагогическое образование англ нем 3++\\история немецкого языка.txt',
    '\\ИМСИТ\\педагогическое образование изобразительное искусство и информатика\\основы проектной деятельности.txt',
    '\\ИМСИТ\\педагогическое образование история и право 3++\\методика обучения немецкий язык.txt',
    '\\ИМСИТ\\педагогическое образование история и право 3++\\основы проектной деятельности.txt',
    '\\ИМСИТ\\педагогическое образование история и право 3++\\практический курс английского языка.txt',
    '\\ИМСИТ\\прикладная информатика\\системы управления хранилищами данных.txt',
    '\\ИМСИТ\\прикладная информатика\\теория вероятностей и математическая статистика.txt',
    '\\ИМСИТ\\прикладная информатика\\физическая культура и спорт общая физическая подготовка.txt',
    '\\ИМСИТ\\реклама и связи с общественностью 3++\\практикум технологиии эффективного общения.txt',
    '\\АлтГУ\\юриспруденция общеправовой\\конституционное право.txt',
    '\\АлтГТУ\\цифровая экономика\\проектирование информационных систем.txt',
]

## Считывание данных

In [2]:
def read_to_df(folder: str, ignore=IGNOREFILES):
    df = pd.DataFrame(columns=['univ', 'prog', 'name', 'text'])
    for file in tqdm(glob.glob('{}/*/*/*.txt'.format(folder))):
        splitted = file.split('\\')
        name = splitted[-1][:-4]
        u = splitted[-3]
        op = splitted[-2]
        if '\\' + u + '\\' + op + '\\' + splitted[-1] in ignore:
            continue
        
        try:
            text = open(file, 'r').read()
        except Exception as e:
            print(e)
            print(file)
            text = open(file, 'r', encoding='utf-8').read()
        
        df = df.append({'univ':u, 'prog':op, 'name':name, 'text':text.lower()}, ignore_index=True)
    return df

In [3]:
df = read_to_df(FILESPATH)
len(df)

  0%|          | 0/3676 [00:00<?, ?it/s]

3663

## Удаление дубликатов

In [4]:
df.drop_duplicates(subset='text', inplace=True)
len(df)

3218

## Лемматизация

In [5]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

from pymystem3 import Mystem
ms = Mystem()

a = ord('а')
DEFAULT_STOP = set(stopwords.words('russian') + [chr(i) for i in range(a, a + 32)] + [_ for _ in range(10)])

In [6]:
def lemmatize(text, regs=None):
    lemmas = ms.lemmatize(text)
    text = ''.join(lemmas).rstrip('\n')
    if regs is not None:
        for reg in regs:
            text = re.sub(reg, '', text)
    return text


def process_text(text, f_lemmatizer=lemmatize, wstop=DEFAULT_STOP,
                 lemm_regs=None, other_regs=None, counter=None):
    
    if f_lemmatizer is not None:
        text = f_lemmatizer(text, lemm_regs)
    
    if other_regs is not None:
        for reg in other_regs:
            text = re.sub(reg, '', text, flags=re.IGNORECASE)
              
    text = [word for word in word_tokenize(text) if word not in wstop]
    if counter is not None:
        counter.update(text)
    return ' '.join(text)

In [7]:
import time
fd_counter = FreqDist()

start = time.time()
df.text = df.text.apply(process_text, counter=fd_counter)
print('Time spent: {} s.'.format(time.time() - start))

Time spent: 2802.084702014923 s.


In [8]:
len(fd_counter)

22144

In [9]:
import json
jsonStr = json.dumps(dict(fd_counter.most_common()), ensure_ascii=False)
file = open('JsonCounter.json', 'w')
file.write(jsonStr)
file.close()

Еще небольшая очистка.

In [10]:
corrected_dict = {}
typo_words = []
for item in sorted(fd_counter.items()):
    if not str.isalpha(item[0][0]):
        typo_words.append(item[0])
        continue
    corrected_dict[item[0]] = item[1]
len(corrected_dict)

21768

In [11]:
d = FreqDist(corrected_dict)
d

FreqDist({'дисциплина': 13067, 'навык': 12679, 'деятельность': 11297, 'метод': 8814, 'основной': 8715, 'знание': 7945, 'система': 7933, 'профессиональный': 7069, 'основа': 6548, 'анализ': 6257, ...})

Установим пороги на число вхождений: верхний 500, нижний -- отсутствует, так как есть некоторые слова, которые встречаются лишь однажды (например, немецкие).

In [12]:
words_most = []
words_least = []
most = 500
least = 1
for w in d.keys():
    if d[w] >= most:
        words_most.append(w)
    elif d[w] <= least:
        words_least.append(w)

upd_wstop = set(words_most + words_least + typo_words)
len(upd_wstop)

9589

In [13]:
fd_counter_trunc = FreqDist()
df_trunc = df.copy()

In [14]:
start = time.time()
df_trunc.text = df_trunc.text.apply(process_text, f_lemmatizer=None, wstop=upd_wstop, counter=fd_counter_trunc)
print('Time spent: {} s.'.format(time.time() - start))

Time spent: 3.74792742729187 s.


In [15]:
len(list(dict(fd_counter_trunc)))

12555

In [16]:
jsonStr_trunc = json.dumps(dict(fd_counter_trunc.most_common()), ensure_ascii=False)
file = open('JsonCounter_trunc.json', 'w')
file.write(jsonStr_trunc)
file.close()

## Запись

In [17]:
def rewrite(folder: str, df):
    for row in df.iterrows():
        r = row[1]
        path = folder + '\\' + r['univ'] + '\\' + r['prog']
        try:
            os.makedirs(path)
        except FileExistsError:
            pass
        
        filename = path + '\\' + r['name'] + '.txt'

        file_ = open(filename, 'w', encoding='ansi')
        try:
            file_.write(r['text'])
        except Exception as e:
            print('FATAL:', e)
            print(row[0], filename)
            file_.close() 
            break
        file_.close()

In [18]:
rewrite(DATAPATH + '\\new-files-lemm', df_trunc)